# Import Required Libraries

In [80]:
import string

from transformers import pipeline, BertTokenizer, BertForMaskedLM, AlbertTokenizer, AlbertForMaskedLM
from transformers.pipelines.fill_mask import FillMaskPipeline
import torch

from spacy.tokens.token import Token
from spacy.tokens.doc import Doc
import editdistance
import pandas as pd
import string

In [81]:
# torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_device = 'cpu'
print(f"Torch Device: {torch_device}")

language = 'fa'

# EN
# model_name = "bert-large-uncased"
# model_name = "bert-base-uncased"

# FA
model_type = 'bert'

# model_name = "HooshvareLab/albert-fa-zwnj-base-v2" # Albert
# model_name = "HooshvareLab/bert-fa-base-uncased" # BERT V2
model_name = "HooshvareLab/bert-fa-zwnj-base" # BERT V3

if model_type == 'bert':
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForMaskedLM.from_pretrained(model_name).to(torch_device) if language == 'en' else BertForMaskedLM.from_pretrained(model_name).to(torch_device)

elif model_type == 'albert':
    tokenizer = AlbertTokenizer.from_pretrained(model_name)
    model = AlbertForMaskedLM.from_pretrained(model_name).to(torch_device) if language == 'en' else AlbertForMaskedLM.from_pretrained(model_name).to(torch_device)


print(f"{language} Model Loaded ...")

unmasker = pipeline("fill-mask", model=model, tokenizer=tokenizer)

Torch Device: cpu
fa Model Loaded ...


In [82]:
# input_text = "The capital of Iran is [MASK]."
# input_text = "Yesterday, I played [MASK] in the [MASK]."
# input_text = "I'm studying [MASK] learning in my computer class."
# input_text = "I'm a very [MASK] player in football."
# input_text = "He drived a [MASK]."
# input_text = "I love playing [MASK]."
# input_text = "He was [MASK] english for the tomorrow [MASK]."

# input_text = "امروز در استادیوم آزادی [ماسک] ملی ایران و روسیه مسایقه می‌دهند."
# input_text = "پس از سال‌ها تلاش رازی موفق به [ماسک] الکل شد. این دانشمند [ماسک] باعث افتخار در تاریخ [ماسک] است."
input_text = "بسیاری از مباحث علوم غیر طبیعی با استفاده از فیزیک دنیای مادی [ماسک] [ماسک] نیست و برای یادگیری باید به فلسفه‌های خاصی [ماسک] کرد."

input_text = input_text.replace("[ماسک]", "[MASK]")

result = unmasker(input_text, top_k=50)
result[1]

[{'score': 0.09131668508052826,
  'token': 3111,
  'token_str': 'پ ذ ی ر',
  'sequence': '[CLS] بسیاری از مباحث علوم غیر طبیعی با استفاده از فیزیک دنیای مادی [MASK] پذیر نیست و برای یادگیری باید به فلسفههای خاصی [MASK] کرد. [SEP]'},
 {'score': 0.03739292919635773,
  'token': 4219,
  'token_str': 'د س ت ر س',
  'sequence': '[CLS] بسیاری از مباحث علوم غیر طبیعی با استفاده از فیزیک دنیای مادی [MASK] دسترس نیست و برای یادگیری باید به فلسفههای خاصی [MASK] کرد. [SEP]'},
 {'score': 0.03577372059226036,
  'token': 35045,
  'token_str': '# # ش ن ا',
  'sequence': '[CLS] بسیاری از مباحث علوم غیر طبیعی با استفاده از فیزیک دنیای مادی [MASK]شنا نیست و برای یادگیری باید به فلسفههای خاصی [MASK] کرد. [SEP]'},
 {'score': 0.023947319015860558,
  'token': 3320,
  'token_str': 'ط ب ی ع ی',
  'sequence': '[CLS] بسیاری از مباحث علوم غیر طبیعی با استفاده از فیزیک دنیای مادی [MASK] طبیعی نیست و برای یادگیری باید به فلسفههای خاصی [MASK] کرد. [SEP]'},
 {'score': 0.020062608644366264,
  'token': 3691,
  'token_s

# Stanza

spaCy's tokenization is non-destructive, so it always represents the original input text and never adds or deletes anything. This is kind of a core principle of the Doc object: you should always be able to reconstruct and reproduce the original input text.

## Setup

In [83]:
import stanza
import spacy
import spacy_stanza

In [150]:
if language == 'fa':
    stanza.install_corenlp()
    stanza.download('fa')
    nlp = spacy_stanza.load_pipeline("fa")

elif language == 'en':
    spacy.prefer_gpu()
    nlp = spacy.load("en_core_web_lg")

else:
    raise ValueError(f"{language} not supported.")


2022-06-05 03:37:07 WARNING: Directory /home/ahur4/stanza_corenlp already exists. Please install CoreNLP to a new directory.


2022-06-05 03:37:07 INFO: Downloading default packages for language: fa (Persian)...
2022-06-05 03:37:08 INFO: File exists: /home/ahur4/stanza_resources/fa/default.zip
2022-06-05 03:37:10 INFO: Finished downloading models and saved to /home/ahur4/stanza_resources.


2022-06-05 03:37:12 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |
| ner       | arman   |

2022-06-05 03:37:12 INFO: Use device: cpu
2022-06-05 03:37:12 INFO: Loading: tokenize
2022-06-05 03:37:12 INFO: Loading: mwt
2022-06-05 03:37:12 INFO: Loading: pos
2022-06-05 03:37:12 INFO: Loading: lemma
2022-06-05 03:37:12 INFO: Loading: depparse
2022-06-05 03:37:12 INFO: Loading: ner
2022-06-05 03:37:12 INFO: Done loading processors!


In [158]:
vocab:set = set(tokenizer.get_vocab().keys())
MASK:str = "[MASK]"

## Correct Lexico Typo

In [173]:
def half_space_case(current_text:str, predicted_text:str):
                if '‌' in current_text:
                    current_text_wo_half_space = current_text.replace('‌', '')
                    return current_text_wo_half_space == predicted_text

def lexico_typo_correction(
        text,
        max_edit_distance_to_length_ratio = 0.45,
        max_edit_distance = 2,
        min_score = 1e-7,
        top_k = 10,
        verbose=False,
):

    while True:
        some_token_corrected = False
        doc = nlp(text)
        for index, current_token in enumerate(doc):
            current_token: Token
            start_char_index: int = current_token.idx
            end_char_index = start_char_index + len(current_token)

            if current_token.text not in vocab:
                if verbose:
                    print("*" * 50)
                    print(f"[{current_token.text}] is not in vocab")

                masked_text = doc.text[:start_char_index] + MASK + doc.text[end_char_index:]

                predicts = unmasker(masked_text, top_k=top_k)

                ### Select Token From Predicts
                predicts = pd.DataFrame(predicts)

                predicts['token_str'] = predicts['token_str'].apply(lambda tk: tk.replace(" ", ""))
                predicts['edit_distance'] = predicts['token_str'].apply(lambda tk: editdistance.eval(current_token.text, tk))
                predicts['edit_distance_to_len_ratio'] = predicts['edit_distance'] / len(current_token.text)

                selected_predicts = predicts[(predicts['edit_distance_to_len_ratio'] <= max_edit_distance_to_length_ratio) & (predicts['edit_distance'] <= max_edit_distance) & (predicts['score'] >= min_score)]

                try:
                    selected_predict = selected_predicts['token_str'].iloc[0]
                except:
                    selected_predict = current_token.text

                if selected_predict != current_token.text and not half_space_case(current_token.text, selected_predict):
                    some_token_corrected = True
                    result_text = masked_text.replace(MASK, selected_predict, 1)
                    text = result_text

                if verbose :
                    print("*" * 50)
                    print(f"Token: {current_token.text}")

                    print("Predicts: \n")
                    print(predicts[['token_str', 'score']])

                    print("Filtered Predicts: \n")
                    print(selected_predicts[['token_str', 'score']])
                    print(f"{current_token.text} -> {selected_predict}")

                    print("SOME TOKEN CORRECTED!!!") \

                if some_token_corrected:
                    break

        if not some_token_corrected:
            break

    if verbose:
        print(f"Result text: \n{text}")

    return text


## Correct Contextual Typo



In [174]:
def contextual_typo_correction(
        text,
        max_edit_distance_to_length_ratio = 0.45,
        max_edit_distance = 2,
        min_score = 1e-7,
        top_k = 10,
        verbose=False,
):

    while True:
        some_token_corrected = False
        doc = nlp(text)
        for index in range(len(doc)):
            current_token: Token = doc[index]

            start_char_index = current_token.idx
            end_char_index = start_char_index + len(current_token)

            masked_text = doc.text[:start_char_index] + MASK + doc.text[end_char_index:]

            predicts = unmasker(masked_text, top_k=top_k)

            ### Select Token From Predicts
            predicts = pd.DataFrame(predicts)

            predicts['token_str'] = predicts['token_str'].apply(lambda tk: tk.replace(" ", ""))
            predicts['edit_distance'] = predicts['token_str'].apply(lambda tk: editdistance.eval(current_token.text, tk))
            predicts['edit_distance_to_len_ratio'] = predicts['edit_distance'] / len(current_token.text)

            selected_predicts = predicts[(predicts['edit_distance_to_len_ratio'] <= max_edit_distance_to_length_ratio) & (predicts['edit_distance'] <= max_edit_distance) & (predicts['score'] >= min_score)]

            try:
                if current_token.text in string.punctuation:
                    selected_predict = selected_predicts['token_str'].iloc[0]

                elif current_token.text.isdigit():
                    selected_predict = current_token.text

                elif current_token.text in selected_predicts['token_str'].values:
                    selected_predict = current_token.text

                else:
                    selected_predict = selected_predicts.sort_values('edit_distance_to_len_ratio')['token_str'].iloc[0]

            except:
                selected_predict = current_token.text

            if selected_predict != current_token.text and not half_space_case(current_token.text, selected_predict):
                some_token_corrected = True
                result_text = masked_text.replace(MASK, selected_predict, 1)
                text = result_text

            if verbose :
                print("*" * 50)
                print(f"Token: {current_token.text}")

                print("Predicts: \n")
                print(predicts[['token_str', 'score']])

                print("Filtered Predicts: \n")
                print(selected_predicts[['token_str', 'score']])
                print(f"{current_token.text} -> {selected_predict}")

                print("SOME TOKEN CORRECTED!!!")

            if some_token_corrected:
                break


        if not some_token_corrected:
            break

    if verbose:
        print(f"Result text: \n{text}")

    return text

# Correction Pipeline Class

In [175]:
class SpellCorrector:

    def __init__(
            self,
            max_edit_distance_to_length_ratio = 0.45,
            max_edit_distance = 2,
            min_score = 1e-7,
            verbose = False,
            top_k = 50
    ):
        self.max_edit_distance_to_length_ratio = max_edit_distance_to_length_ratio
        self.max_edit_distance = max_edit_distance
        self.min_score = min_score
        self.verbose = verbose
        self.top_k = top_k

    def _lexico_typo_correction(self, text):
        return lexico_typo_correction(text, self.max_edit_distance_to_length_ratio, self.max_edit_distance, self.min_score, self.top_k, self.verbose, )

    def _contextual_typo_correction(self, text):
        return contextual_typo_correction(text, self.max_edit_distance_to_length_ratio, self.max_edit_distance, self.min_score, self.top_k, self.verbose,)

    def correction_pipeline(self, text):
        print("Lexico Correction ...") if self.verbose else print()
        corrected_text = self._lexico_typo_correction(text)

        print("Contextual Correction ...") if self.verbose else print()
        corrected_text = self._contextual_typo_correction(corrected_text)
        return corrected_text

    def __call__(self, text, *args, **kwargs):
        return self.correction_pipeline(text)


# Test On Sample Texts

In [177]:
MAX_EDIT_DISTANCE_TO_LEN_RATIO = 0.4
MAX_EDIT_DISTANCE = 2
MIN_SCORE = 0
TOP_K = 250
VERBOSE = True

if language == 'en':
    input_text = "The capitan of Iran is tehran."
    input_text = "i am speeking english very wall."
    input_text = "He was stadying english for the finall exam."
    # text = "I'm studying [MASK] learning in my computer class."
    # text = "I'm a very [MASK] player in football."
    input_text = "He drove a cat."
    # text = "do you want to watch tv."
    # text = "I love playing [MASK]."

if language == 'fa':
    input_text = "امروز در استادیوم آزادی تیم ملی ایران و روسیه مسایقه می‌دهند."
    input_text = "پس از سال‌ها تلاش رازی موفق به کسف الکل شد. این دانشمند تیرانی باعث افتخار در تاریخ کور است."
    input_text = "هفته آینده احتمالا توافق بسته‌ای امضا می‌شود."
    input_text = "اهل کدام کشور هستی."
    input_text = "سن شما چقدر است."
    input_text = "وقتی قیمت گوست قرمز یا صفید در کشورهای دیگر بیشتر شده است، ممکن است در جیران هم گرا شود."
    input_text = "در هفته گذشته قیمت تلا تغییر چندانی نداشت، و در همان محدوده 1850 دلاری کار خود را به پایان رساند. "
    input_text = "هدف از زندگانی چیست!"
    input_text = "همه رأس ساعت 3 در جلسه حاضر باشند."

    input_text = "بر اساس مسوبه سران قوا، معاملات فردایی طلا همانند معاملات فردایی ارض، ممنوع و غیرقانونی شناخته شد و فعالان این بازار به جرم اخلال اقتصادی، تحت پیگرد قرار خواهند گرفت. در نتیجه تانک مرکزی در بازار فردایی مداخله نخواهد کرد"

    input_text = """
        با نزدیک شدن قیمت دار غیر رسمی به سفف خود در روز قبل، تحلیلگران در بازار برای هفته بعد هشدار میدادند که باید احطیاط کرد و اقدامات امنیتی در بازار افزایش خواهد یافت.
    """

    input_text = """
    با تولانی شدن جنگ روسیه و اوکراین و سهم قابل توجهی که این دو کشور در تأمین کندم جهان داشتند، بازار کندم با نوسانات زیادی مواجه شد و قیمت محصولاتی که مواد اولیه‌شان کندم بود، در همه جای جهان افزایش یافت.
    """


if language == 'en':
    input_text = input_text.lower()


input_text = input_text.strip()

print(f"normalized text: {input_text}")

spell_corrector = SpellCorrector(MAX_EDIT_DISTANCE_TO_LEN_RATIO, MAX_EDIT_DISTANCE, MIN_SCORE, VERBOSE, TOP_K)

doc = nlp(input_text)

from spacy import displacy
displacy.render(doc, style="dep")

result = " ".join([spell_corrector(sentence.text) for sentence in doc.sents])
result


normalized text: با تولانی شدن جنگ روسیه و اوکراین و سهم قابل توجهی که این دو کشور در تأمین کندم جهان داشتند، بازار کندم با نوسانات زیادی مواجه شد و قیمت محصولاتی که مواد اولیه‌شان کندم بود، در همه جای جهان افزایش یافت.


Lexico Correction ...
**************************************************
[تولانی] is not in vocab
**************************************************
Token: تولانی
Predicts: 

    token_str     score
0        شروع  0.238026
1       نزدیک  0.208401
2        تمام  0.041988
3      طولانی  0.028183
4        بزرگ  0.025659
..        ...       ...
495     اعمال  0.000070
496     نایاب  0.000069
497     پیاده  0.000069
498    سنگینی  0.000069
499      ژاپن  0.000069

[500 rows x 2 columns]
Filtered Predicts: 

    token_str     score
3      طولانی  0.028183
188    طوفانی  0.000333
192    دورانی  0.000328
تولانی -> طولانی
SOME TOKEN CORRECTED!!!
**************************************************
[تأمین] is not in vocab
**************************************************
Token: تأمین
Predicts: 

    token_str     score
0       بازار  0.744573
1    بازارهای  0.058623
2         بخش  0.011134
3      اقتصاد  0.010307
4       برابر  0.008009
..        ...       ...
495  تکنولوژی  0.000024
496     مناف

'با طولانی شدن جنگ روسیه و اوکراین و سهم قابل توجهی که این دو کشور در تامین گندم جهان داشتند، بازار گندم با نوسانات زیادی مواجه شد و قیمت محصولاتی که مواد اولیه\u200cشان گندم بود، در همه جای جهان افزایش یافت.'

In [113]:
"تلا" in vocab

True

In [91]:
# text = "The capital of Iran is [MASK]."
# text = "Yesterday, I played [MASK] in the [MASK]."
# text = "I'm studying [MASK] learning in my computer class."
# text = "I'm a very [MASK] player in football."
# text = "He drived a [MASK]."
# text = "I love playing [MASK]."
# text = "I am [MASK] english very [MASK]. "
#
# # text = "امروز در استادیوم آزادی، تیم ملی [MASK] ایران و سوریه مسابقه می‌دهند."
# # text = "امروز در استادیوم آزادی [ماسک] ملی ایران و روسیه مسایقه می‌دهند."
# # text = "پس از سال‌ها تلاش رازی موفق به [ماسک] الکل شد. این دانشمند تیرانی باعث افتخار در تاریخ [ماسک] است."
# # text = "اهل کدام کشور هستی[ماسک]"
# # text = text.replace("[ماسک]", "[MASK]")
#
# unmasker(text, )

In [92]:
# tokens = tokenizer.convert_ids_to_tokens(tokenizer(text)['input_ids'])
# tokens
# tokenizer.convert_tokens_to_string(tokens)


# ContextualSpellCheck Lib

In [93]:
# import spacy
# import contextualSpellCheck
#
# nlp = spacy.load('en_core_web_lg')
# contextualSpellCheck.add_to_pipe(nlp)

In [94]:
# text = 'i am speaking english very wall.'
# doc = nlp(text)
#
# print(doc._.performed_spellCheck) #Should be True
# print(doc._.outcome_spellCheck) #Income was $9.4 million compared to the prior year of $2.7 million.

In [95]:
# # tokens = tokenizer.convert_ids_to_tokens(tokenizer(text)['input_ids'])
# # tokenizer.convert_tokens_to_string(tokens)
#
# # tokenizer.get_vocab()
#
# text = "The capitan of Iran is tehran."
# text = "i am speeking english very wall."
# # text = "Yesterday, I played [MASK] in the [MASK]."
# # text = "I'm studying [MASK] learning in my computer class."
# # text = "I'm a very [MASK] player in football."
# # text = "He drove a cat."
# # text = "do you want to watch tv."
# # text = "I love playing [MASK]."
#
# text = text.lower()
# vocab: dict = tokenizer.get_vocab()
# MASK = "[MASK]"
#
# MAX_EDIT_DISTANCE_TO_LEN_RATIO = 0.5
# MIN_SCORE = 0.01
#
# ### Correct Lexico Typo
# print("\n\nCorrect Lexico Typo:\n\n")
# print(f"Text: {text}\n\n")
# while True:
#     some_token_corrected = False
#     doc = nlp(text)
#     for index in range(len(doc)):
#         current_token: Token = doc[index]
#         start_char_index = current_token.idx
#         end_char_index = start_char_index + len(current_token)
#
#         if current_token.text not in vocab:
#             print(f"[{current_token.text}] is not in vocab")
#
#             tokens = list(doc)
#             print(f"Tokens: {tokens}")
#
#             masked_text = doc.text[:start_char_index] + MASK + doc.text[end_char_index:]
#
#             predicts = unmasker(masked_text)
#
#             ### Select Token From Predicts
#             predicts = pd.DataFrame(predicts)
#
#
#             predicts['token_str'] = predicts['token_str'].apply(lambda tk: tk.replace(" ", ""))
#             predicts['edit_distance_to_len_ratio'] = predicts['token_str'].apply(lambda tk: editdistance.eval(current_token.text, tk) / len(current_token.text))
#
#             print("Predicts: \n")
#             print(predicts)
#
#             predicts = predicts[predicts['edit_distance_to_len_ratio'] <= MAX_EDIT_DISTANCE_TO_LEN_RATIO]
#
#             print("Filtered Predicts: \n")
#             print(predicts)
#
#             selected_predict = predicts['token_str'].iloc[0]
#             ###
#
#             result_text = masked_text.replace(MASK, selected_predict, 1)
#
#             print(f"{current_token.text} -> {selected_predict}")
#             print(f"Text: {result_text}\n\n")
#
#             text = result_text
#             some_token_corrected = True
#             break
#
#     if not some_token_corrected:
#         break

In [96]:
# # tokens = tokenizer.convert_ids_to_tokens(tokenizer(text)['input_ids'])
# # tokenizer.convert_tokens_to_string(tokens)
#
# # tokenizer.get_vocab()
#
# text = "The capitan of Iran is tehran."
# text = "i am speeking english very wall."
# # text = "Yesterday, I played [MASK] in the [MASK]."
# # text = "I'm studying [MASK] learning in my computer class."
# # text = "I'm a very [MASK] player in football."
# # text = "He drove a cat."
# # text = "do you want to watch tv."
# # text = "I love playing [MASK]."
#
# text = text.lower()
# vocab: dict = tokenizer.get_vocab()
# MASK = "[MASK]"
#
# MAX_EDIT_DISTANCE_TO_LEN_RATIO = 0.5
# MIN_SCORE = 0.01
#
# ### Correct Lexico Typo
# print("\n\nCorrect Lexico Typo:\n\n")
# print(f"Text: {text}\n\n")
# while True:
#     some_token_corrected = False
#     doc = nlp(text)
#     for index in range(len(doc)):
#         current_token: Token = doc[index]
#         start_char_index = current_token.idx
#         end_char_index = start_char_index + len(current_token)
#
#         if current_token.text not in vocab:
#             print(f"[{current_token.text}] is not in vocab")
#
#             tokens = list(doc)
#             print(f"Tokens: {tokens}")
#
#             masked_text = doc.text[:start_char_index] + MASK + doc.text[end_char_index:]
#
#             predicts = unmasker(masked_text)
#
#             ### Select Token From Predicts
#             predicts = pd.DataFrame(predicts)
#
#
#             predicts['token_str'] = predicts['token_str'].apply(lambda tk: tk.replace(" ", ""))
#             predicts['edit_distance_to_len_ratio'] = predicts['token_str'].apply(lambda tk: editdistance.eval(current_token.text, tk) / len(current_token.text))
#
#             print("Predicts: \n")
#             print(predicts)
#
#             predicts = predicts[predicts['edit_distance_to_len_ratio'] <= MAX_EDIT_DISTANCE_TO_LEN_RATIO]
#
#             print("Filtered Predicts: \n")
#             print(predicts)
#
#             selected_predict = predicts['token_str'].iloc[0]
#             ###
#
#             result_text = masked_text.replace(MASK, selected_predict, 1)
#
#             print(f"{current_token.text} -> {selected_predict}")
#             print(f"Text: {result_text}\n\n")
#
#             text = result_text
#             some_token_corrected = True
#             break
#
#     if not some_token_corrected:
#         break